In [1]:
suppressPackageStartupMessages(library(MouseGastrulationData))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(Matrix))

In [2]:
getwd()

[1] "/rds/project/rds-SDzz0CATGms/users/bt392/12_Eomes_T_Mixl1/T"

In [3]:
io = list()
io$outdir = '/rds/project/rds-SDzz0CATGms/users/bt392/12_Eomes_T_Mixl1/T/data/'
io$genemetadata = "/rds/project/rds-SDzz0CATGms/users/bt392/atlasses/Mmusculus_genes_BioMart.87.txt"

In [4]:
T_E85.sce = TChimeraData(type = c("processed"), samples = c(1:2, 5:10))

snapshotDate(): 2022-10-31

see ?MouseGastrulationData and browseVignettes('MouseGastrulationData') for documentation

loading from cache

see ?MouseGastrulationData and browseVignettes('MouseGastrulationData') for documentation

loading from cache

see ?MouseGastrulationData and browseVignettes('MouseGastrulationData') for documentation

loading from cache

see ?MouseGastrulationData and browseVignettes('MouseGastrulationData') for documentation

loading from cache

see ?MouseGastrulationData and browseVignettes('MouseGastrulationData') for documentation

loading from cache

see ?MouseGastrulationData and browseVignettes('MouseGastrulationData') for documentation

loading from cache

see ?MouseGastrulationData and browseVignettes('MouseGastrulationData') for documentation

loading from cache

see ?MouseGastrulationData and browseVignettes('MouseGastrulationData') for documentation

loading from cache

see ?MouseGastrulationData and browseVignettes('MouseGastrulationData') for document

In [5]:
T_E85.sce

class: SingleCellExperiment 
dim: 29453 30758 
metadata(0):
assays(1): counts
rownames(29453): ENSMUSG00000051951 ENSMUSG00000089699 ...
  ENSMUSG00000095742 tomato-td
rowData names(2): ENSEMBL SYMBOL
colnames(30758): cell_1 cell_2 ... cell_31676 cell_31677
colData names(13): cell barcode ... somite.subct.mapped sizeFactor
reducedDimNames(2): pca.corrected.E7.5 pca.corrected.E8.5
mainExpName: NULL
altExpNames(0):

In [35]:
gene_metadata = fread(io$genemetadata)[, c('ens_id', 'symbol')] %>%
    rbind(., data.table(ens_id='tomato-td', symbol='tomato-td')) %>% 
    .[ens_id %in% rownames(T_E85.sce)] %>% 
    .[symbol!='']

T_E85.sce = T_E85.sce[rownames(T_E85.sce) %in% gene_metadata$ens_id]

In [36]:
rownames(T_E85.sce) = gene_metadata[match(rownames(T_E85.sce), ens_id), symbol]

In [37]:
summary(rownames(T_E85.sce)[order(rownames(T_E85.sce))] == gene_metadata$symbol[order(gene_metadata$symbol)])

   Mode    TRUE 
logical   27995 

In [38]:
meta = as.data.table(colData(T_E85.sce), keep.rownames=F) #%>% setnames('rn', 'cell')

Warning message in .local(x, row.names, optional, ...):
“Arguments in '...' ignored”


In [39]:
lapply(unique(meta$sample), function(x){ # unique(meta$sample)
    tmp = meta[sample==x]
    tmp.sce = T_E85.sce[,tmp$cell]
    
    tmp.outdir = paste0(io$outdir, 'sample_', x, '/outs/filtered_feature_bc_matrix')
    dir.create(tmp.outdir, recursive = TRUE, showWarnings = FALSE)
    fwrite(tmp[,'barcode'], file.path(tmp.outdir, 'barcodes.tsv.gz'), col.names=F)
    
    features =  gene_metadata[match(rownames(tmp.sce), symbol), c('ens_id', 'symbol')] %>%
        .[, gene_expression := 'Gene Expression']
    stopifnot(features$symbol == rownames(tmp.sce))
    fwrite(features, file.path(tmp.outdir, 'features.tsv.gz'), col.names=F)

    writeMM(counts(tmp.sce), file.path(tmp.outdir, 'matrix.mtx.gz'))
    cat(paste0('Sample ', x, ' done \n'))
})

Sample 1 done 
Sample 2 done 
Sample 5 done 
Sample 6 done 
Sample 7 done 
Sample 8 done 
Sample 9 done 
Sample 10 done 


[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL